In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [91]:
bread = pd.read_csv("/content/drive/MyDrive/dataminassi/assignment4/bread.csv")
bread
bread = bread.drop_duplicates()
new = bread['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [92]:
bread['Date'] = new[0]
bread['Time'] = new[1]
bread['Transaction'] = new[2]
bread['Item'] = new[3]
bread[['Date', 'Time', 'Transaction', 'Item']].head(10)

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread
10,2016-10-30,10:16:55,6,Medialuna


In [93]:
transaction = pd.crosstab(index= bread['Transaction'], columns= bread['Item'])
# transaction

## removing unwanted "NONE"
transaction = transaction.drop(['NONE'], axis = 1)

In [94]:
def APRIORI_MY(data, min_support=0.04,  max_length = 4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {} 
    L = list(data.columns)
    
    # Step 2: 
    #generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = set(combinations(L,i))
        
    # Reset "L" for next ith iteration
        L =set()     
    # Step 3: 
        #iterate through each item in "c"
        for j in list(c):
            #print(j)
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            if sup > min_support:
                #print(sup, j)
                support[j] = sup
                
                # Appending frequent itemset in list "L", already reset list "L" 
                L = list(set(L) | set(j))
        
    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
    return(result)

In [95]:
## finding frequent itemset with min support = 4%
my_freq_itemset = APRIORI_MY(transaction, 0.04, 3)
my_freq_itemset.sort_values(by = 'Support', ascending = False)

,Items,Support
2,"(Coffee,)",0.475081
5,"(Bread,)",0.324940
3,"(Tea,)",0.141643
4,"(Cake,)",0.103137
12,"(Bread, Coffee)",0.089393
6,"(Pastry,)",0.085510
0,"(Sandwich,)",0.071346
8,"(Medialuna,)",0.061379
1,"(Hot chocolate,)",0.057916
10,"(Coffee, Cake)",0.054349


In [96]:
def ASSOCIATION_RULE_MY(df, min_threshold=0.5):
    import pandas as pd
    from itertools import permutations
    
    # STEP 1:
    #creating required varaible
    support = pd.Series(df.Support.values, index=df.Items).to_dict()
    data = []
    L= df.Items.values
    
    # Step 2:
    #generating rule using permutation
    p = list(permutations(L, 2))
    
    # Iterating through each rule
    for i in p:
        
        # If LHS(Antecedent) of rule is subset of RHS then valid rule.
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            #print(i, conf)
            if conf > min_threshold:
                #print(i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

         
    # STEP 3:
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [97]:
my_rule = ASSOCIATION_RULE_MY(my_freq_itemset, 0.5)
newdf = my_rule[my_rule['support'] > 0.02]
newdf

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
0,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667
1,"(Pastry,)","(Coffee,)",0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
